In [1]:
# Bernoulli-Poisson link CP decomposition
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

In [2]:
# 連続した日付を取得する関数
def daterange(_start, _end):
    for n in range((_end - _start).days):
        yield _start + timedelta(n)
        
# 切断ポアソン分布を生成する関数
def rtpois(mu, a, b, n):
    FA = scipy.stats.poisson.cdf(a, mu)
    FB = scipy.stats.poisson.cdf(b, mu)
    return np.array(scipy.stats.poisson.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu), dtype="int")

# 多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

In [3]:
# データの定義
k = 15
hh = 5000
item = 3000
element = 500
Lambda = np.random.gamma(50.0, 1/0.2, hh)
pt1 = np.random.poisson(Lambda, hh)
hhpt = np.sum(pt1)
k_vec = np.repeat(1.0, k)

In [4]:
# idとインデックスを定義
# idの定義
d_id1 = np.repeat(np.arange(hh), pt1)
pt_id1 = np.array(list(itertools.chain(*[np.array(range(pt1[i]), dtype="int") for i in range(hh)])))

#インデックスを定義
d_list1 = [i for i in range(hh)]
d_vec1 = [i for i in range(hh)]
for i in range(hh):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int")
    d_vec1[i] = np.repeat(1.0, d_list1[i].shape[0])

In [5]:
# itemとelementを選択
#トピックを生成
topic = 30
theta_topic = np.random.dirichlet(np.repeat(0.2, topic), hh)
phi_item = np.random.dirichlet(np.repeat(0.1, item), topic)
phi_element = np.random.dirichlet(np.repeat(0.1, element), topic)
z = np.array(rmnom(theta_topic[d_id1, ], hhpt, topic, 0), dtype="int16")

# 多項分布からplaceを生成
item_id1 = np.repeat(0, hhpt)
element_id1 = np.repeat(0, hhpt)
for i in range(hh):
    index = d_list1[i]
    item_id1[index] = rmnom(phi_item[z[index], ], pt1[i], item, 0)
    element_id1[index] = rmnom(phi_element[z[index], ], pt1[i], element, 0)
    
# インデックスを定義
item_list1 = [i for i in range(item)]
element_list1 = [i for i in range(element)]
item_vec1 = [i for i in range(item)]
element_vec1 = [i for i in range(element)]
item_n1 = np.repeat(0, item)
element_n1 = np.repeat(0, element)
for i in range(item):
    item_list1[i] = np.array(np.where(item_id1==i)[0], dtype="int")
    item_vec1[i] = np.repeat(1.0, item_list1[i].shape[0])
    item_n1[i] = item_list1[i].shape[0]
for i in range(element):
    element_list1[i] = np.array(np.where(element_id1==i)[0], dtype="int")
    element_vec1[i] = np.repeat(1.0, element_list1[i].shape[0])
    element_n1[i] = element_list1[i].shape[0]

In [6]:
# 応答変数を生成
# パラメータを生成
theta_u = np.random.gamma(0.2, 2.0, k*hh).reshape(hh, k)
theta_v = np.random.gamma(0.2, 2.0, k*item).reshape(item, k)
theta_e = np.random.gamma(0.5, 1.0, k*element).reshape(element, k)
thetat_u = theta_u.copy(); thetat_v = theta_v.copy(); thetat_e = theta_e.copy()

# Bernoulli Poisson linkからbinaryデータを生成
mu = np.dot(theta_u[d_id1, ] * theta_v[item_id1, ] * theta_e[element_id1, ], k_vec)
freq = np.random.poisson(mu, hhpt)
x = np.array(freq > 0, dtype="int")
index_x = np.array(np.where(x==1)[0], dtype="int")
N = index_x.shape[0]

In [7]:
# 新しいidとインデックスを定義
# idを定義
d_id2 = d_id1[index_x]
item_id2 = item_id1[index_x]
element_id2 = element_id1[index_x]

# インデックスを定義
d_list2 = [i for i in range(hh)]
item_list2 = [i for i in range(item)]
element_list2 = [i for i in range(element)]
d_vec2 = [i for i in range(hh)]
item_vec2 = [i for i in range(item)]
element_vec2 = [i for i in range(element)]
pt2 = np.repeat(0, hh)
item_n2 = np.repeat(0, item)
element_n2 = np.repeat(0, element)

for i in range(hh):
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int")
    d_vec2[i] = np.repeat(1.0, d_list2[i].shape[0])
    pt2[i] = d_list2[i].shape[0]
pt_id2 = np.array(list(itertools.chain(*[np.array(range(pt2[i]), dtype="int") for i in range(hh)])))
for i in range(item):
    item_list2[i] = np.array(np.where(item_id2==i)[0], dtype="int")
    item_vec2[i] = np.repeat(1.0, item_list2[i].shape[0])
    item_n2[i] = item_list2[i].shape[0]
for i in range(element):
    element_list2[i] = np.array(np.where(element_id2==i)[0], dtype="int")
    element_vec2[i] = np.repeat(1.0, element_list2[i].shape[0])
    element_n2[i] = element_list2[i].shape[0]

In [14]:
# Bernoulli-Poisson link CP decompositionのパラメータを推定
# MCMCの設定
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 50
freq_y = freq[index_x]

# 事前分布の定義
alpha1 = 0.25; beta1 = 2.5
alpha2 = 0.25; beta2 = 2.5
alpha3 = 0.25; beta3 = 2.5
s0 = 0.5; v0 = 0.5

In [15]:
# パラメータの真値
# モデルパラメータの定義
theta_u = thetat_u.copy()
theta_v = thetat_v.copy()
theta_e = thetat_e.copy()

# 期待値の真値
mu_deploy = theta_u[d_id1, ] * theta_v[item_id1, ] * theta_e[element_id1, ]
mu = np.dot(mu_deploy, k_vec)

In [16]:
# パラメータの初期値
# パラメータを生成
theta_u = np.random.gamma(3.0, 0.2, k*hh).reshape(hh, k)
theta_v = np.random.gamma(3.0, 0.2, k*item).reshape(item, k)
theta_e = np.random.gamma(3.0, 0.2, k*element).reshape(element, k)

# 期待値の初期値
mu_deploy = theta_u[d_id1, ] * theta_v[item_id1, ] * theta_e[element_id1, ]
mu = np.dot(mu_deploy, k_vec)

In [17]:
# パラメータの格納用配列
THETA_U = np.zeros((hh, k, int(R/keep)))
THETA_V = np.zeros((item, k, int(R/keep)))
THETA_E = np.zeros((element, k, int(R/keep)))

In [18]:
# 対数尤度の基準値
# 1パラメータモデルの対数尤度
Prob = np.mean(x)
LLst = np.sum(x*np.log(Prob) + (1-x)*np.log(1-Prob))
print(LLst)

# 真値での対数尤度
mu_deploy = thetat_u[d_id1, ] * thetat_v[item_id1, ] * thetat_e[element_id1, ]
mut = np.dot(mu_deploy, k_vec)
Prob = 1 - np.exp(-mut)
Prob[Prob==1.0] = 0.999999999; Prob[Prob==0.0] = np.power(10.0, -10)
LLbest = np.sum(x*np.log(Prob) + (1-x)*np.log(1-Prob))
print(LLbest)

-838625.5799470674
-516090.5144926023


In [19]:
# ギブスサンプリングでパラメータをサンプリング
for rp in range(R):
    
    # Bernoulli-Poisson linkで潜在頻度を生成
    # 期待値を定義
    mu_deploy = theta_u[d_id2, ] * theta_v[item_id2, ] * theta_e[element_id2, ]
    mu = np.dot(mu_deploy, k_vec)

    # 切断ポアソン分布から潜在頻度を生成
    xy = np.repeat(0, hhpt)
    y = rtpois(mu, 0, np.inf, N)
    xy[index_x] = y
    
    
    # ユーザーの特徴行列を更新
    # 補助変数を定義
    Lambda = mu_deploy / mu[:, np.newaxis]
    lambda_y = y[:, np.newaxis] * Lambda
    lambda_h = theta_v[item_id1, ] * theta_e[element_id1, ]
    
    # 事後分布のパラメータ
    w1 = np.zeros((hh, k))
    w2 = np.zeros((hh, k))
    for i in range(hh):
        index1 = d_list1[i]; index2 = d_list2[i]
        w1[i, ] = np.dot(d_vec2[i].T, lambda_y[index2, ]) + alpha1
        w2[i, ] = np.dot(d_vec1[i].T, lambda_h[index1, ]) + beta1
    
    # ガンマ分布よりパラメータをサンプリング
    theta_u = np.random.gamma(w1, 1/w2)
    mu_deploy = theta_u[d_id2, ] * theta_v[item_id2, ] * theta_e[element_id2, ]
    mu = np.dot(mu_deploy, k_vec)

    
    # itemの特徴行列を更新
    # 補助変数を定義
    Lambda = mu_deploy / mu[:, np.newaxis]
    lambda_y = y[:, np.newaxis] * Lambda
    lambda_h = theta_u[d_id1, ] * theta_e[element_id1, ]
    
    # 事後分布のパラメータ
    w1 = np.zeros((item, k))
    w2 = np.zeros((item, k))
    for i in range(item):
        index1 = item_list1[i]; index2 = item_list2[i]
        w1[i, ] = np.dot(item_vec2[i].T, lambda_y[index2, ]) + alpha2
        w2[i, ] = np.dot(item_vec1[i].T, lambda_h[index1, ]) + beta2
    
    # ガンマ分布よりパラメータをサンプリング
    theta_v = np.random.gamma(w1, 1/w2)
    mu_deploy = theta_u[d_id2, ] * theta_v[item_id2, ] * theta_e[element_id2, ]
    mu = np.dot(mu_deploy, k_vec)

    
    # itemの特徴行列を更新
    # 補助変数を定義
    Lambda = mu_deploy / mu[:, np.newaxis]
    lambda_y = y[:, np.newaxis] * Lambda
    lambda_h = theta_u[d_id1, ] * theta_v[item_id1, ]
    
    # 事後分布のパラメータ
    w1 = np.zeros((element, k))
    w2 = np.zeros((element, k))
    for i in range(element):
        index1 = element_list1[i]; index2 = element_list2[i]
        w1[i, ] = np.dot(element_vec2[i].T, lambda_y[index2, ]) + alpha3
        w2[i, ] = np.dot(element_vec1[i].T, lambda_h[index1, ]) + beta3
    
    # ガンマ分布よりパラメータをサンプリング
    theta_e = np.random.gamma(w1, 1/w2)
    
    
    # サンプリング結果の格納と表示
    if rp%keep==0:
        # サンプリング結果の格納
        mkeep = int(rp/keep)
        THETA_U[:, :, mkeep] = theta_u
        THETA_V[:, :, mkeep] = theta_v
        THETA_E[:, :, mkeep] = theta_e
    
    if rp%disp==0:
        # 対数尤度を更新
        mu_deploy = theta_u[d_id1, ] * theta_v[item_id1, ] * theta_e[element_id1, ]
        mu = np.dot(mu_deploy, k_vec)
        Prob = 1 - np.exp(-mu)
        Prob[Prob==1.0] = 0.999999999; Prob[Prob==0.0] = np.power(10.0, -10)
        LL = np.sum(x*np.log(Prob) + (1-x)*np.log(1-Prob))

        # サンプリング結果の表示
        print(rp)
        print([np.sum(y), np.sum(freq)])
        print(np.round([LL, LLst, LLbest], 1))

0
[1695412, 1548869]
[-1010443.3  -838625.6  -516090.5]
50
[809558, 1548869]
[-658529.5 -838625.6 -516090.5]
100
[1056816, 1548869]
[-561478.8 -838625.6 -516090.5]
150
[1207130, 1548869]
[-528699.4 -838625.6 -516090.5]
200
[1275017, 1548869]
[-513300.3 -838625.6 -516090.5]
250
[1284996, 1548869]
[-513143.2 -838625.6 -516090.5]
300
[1281943, 1548869]
[-512902.7 -838625.6 -516090.5]
350
[1284139, 1548869]
[-512717.2 -838625.6 -516090.5]
400
[1293295, 1548869]
[-512921.4 -838625.6 -516090.5]
450
[1281693, 1548869]
[-512970.7 -838625.6 -516090.5]
500
[1280230, 1548869]
[-512971.7 -838625.6 -516090.5]
550
[1288903, 1548869]
[-513081.2 -838625.6 -516090.5]
600
[1284459, 1548869]
[-512877.1 -838625.6 -516090.5]
650
[1286664, 1548869]
[-513098.6 -838625.6 -516090.5]
700
[1288267, 1548869]
[-512639.  -838625.6 -516090.5]
750
[1286444, 1548869]
[-512962.1 -838625.6 -516090.5]
800
[1286731, 1548869]
[-512989.3 -838625.6 -516090.5]
850
[1281259, 1548869]
[-513026.  -838625.6 -516090.5]
900
[128516